In [128]:
import pandas as pd

# source: https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm
data = [["black", 0, 0, 990],
["black", 0, 1, 805],
["black", 1, 0, 532],
["black", 1, 1, 1369],
["white", 0, 0, 1139],
["white", 0, 1, 349],
["white", 1, 0, 461],
["white", 1, 1, 505]]

df = pd.DataFrame(data, columns=["race", "outcome", "prediction", "count"])

# Fairness in ProPublica's "Machine Bias"

An illustration of incompatable fairness metrics on the ProPublia [Machine Bias](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing). COMPAS is shown to have equal accuracy and calibration for white and black subgroups, but very unequal false positive rates.

Later work showed that accuracy, calibration, and false positive rates cannot be equal if the base rates are unequal.

### Data

Provided in a [methodology piece](https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm) accompanying the original article. 

## Pivot tables

White parolees

In [2]:
pd.pivot_table( df[df["race"]=="white"], values="count", index="outcome", columns="prediction" )

prediction,0,1
outcome,,
0,1139,349
1,461,505


Black parolees

In [3]:
pd.pivot_table( df[df["race"]=="black"], values="count", index="outcome", columns="prediction" )

prediction,0,1
outcome,,
0,990,805
1,532,1369


## Accuracy

What fraction of the time did COMPAS get it right?

In [4]:
total_correct_predictions = df[df.outcome==df.prediction]["count"].sum()
print( "Total correct predictions: ", total_correct_predictions )

Total correct predictions:  4003


In [5]:
total = df["count"].sum()
print( "Total predictions: ", total )

Total predictions:  6150


In [6]:
accuracy = total_correct_predictions / total
print( f"Accuracy: {accuracy*100:0.2f}%" )

Accuracy: 65.09%


### White

In [7]:
df_white = df[df["race"]=="white"]

In [8]:
total_correct_predictions_white = df_white[df_white.outcome==df_white.prediction]["count"].sum()
print( "Total correct predictions, white: ", total_correct_predictions_white )

Total correct predictions, white:  1644


In [9]:
total_white = df_white["count"].sum()
print( "Total predictions, white: ", total_white )

Total predictions, white:  2454


In [10]:
accuracy_white = total_correct_predictions_white / total_white
print( f"Accuracy, white: {accuracy_white*100:0.2f}%" )

Accuracy, white: 66.99%


### Black

In [11]:
df_black = df[df["race"]=="black"]

In [12]:
total_correct_predictions_black = df_black[df_black.outcome==df_black.prediction]["count"].sum()
print( "Total correct predictions, black: ", total_correct_predictions_black )

Total correct predictions, black:  2359


In [13]:
total_black = df_black["count"].sum()
print( "Total predictions, black: ", total_black )

Total predictions, black:  3696


In [14]:
accuracy_black = total_correct_predictions_black / total_black
print( f"Accuracy, black: {accuracy_black*100:0.2f}%" )

Accuracy, black: 63.83%


The accuracy for black and white subgroups are comparable; though the accuracy is a few points higher for the white subgroup.

## Calibration across groups; meaning of a positive prediction

What is the score meaning? Is it different for different groups?

Chance of recidivism

In [36]:
total_recidivate = df[df["outcome"]==1]["count"].sum()

In [37]:
print(f"Overall chance of recidivating: {100*total_recidivate / total:0.2f}%")

Overall chance of recidivating: 131.88%


In [38]:
df_pred_pos = df[df["prediction"]==1]
df_pred_pos_total = df_pred_pos["count"].sum()
df_pred_pos_true = df_pred_pos[ df_pred_pos["outcome"]==1 ]["count"].sum()

print( f"Chance of recidivating for positive prediction (ie, meaning of positive prediction), overall: {100*df_pred_pos_true / df_pred_pos_total:0.2f}%" )

Chance of recidivating for positive prediction (ie, meaning of positive prediction), overall: 61.89%


### White subgroup

In [19]:
dfp = df[ ((df["race"]=="white") & (df["prediction"]==1)) ]
dfp

,race,outcome,prediction,count
5,white,0,1,349
7,white,1,1,505


In [23]:
total = dfp["count"].sum()
total

854

In [24]:
true_pos = dfp[dfp["outcome"]==1].iloc[0]["count"]
true_pos

505

In [28]:
print( f"Meaning of positive prediction, white subgroup: {100*true_pos/total:0.1f}% recidivism" )

Meaning of positive prediction, white subgroup: 59.1% recidivism


### Black subgroup

In [31]:
dfp = df[ ((df["race"]=="black") & (df["prediction"]==1)) ]
dfp

,race,outcome,prediction,count
1,black,0,1,805
3,black,1,1,1369


In [32]:
total = dfp["count"].sum()
total

2174

In [33]:
true_pos = dfp[dfp["outcome"]==1].iloc[0]["count"]
true_pos

1369

In [35]:
print( f"Meaning of positive prediction, black subgroup: {100*true_pos/total:0.1f}% recidivism" )

Meaning of positive prediction, black subgroup: 63.0% recidivism


The meaning of a positive prediction is within a few points for each subgroup; each subgroup is calibrated.

## Meaning of a negative prediction

In [45]:
df_pred_neg = df[df["prediction"]==0]
df_pred_neg_total = df_pred_neg["count"].sum()
df_pred_neg_true = df_pred_neg[ df_pred_neg["outcome"]==1 ]["count"].sum()

print( f"Chance of recidivating for negative prediction (ie, meaning of negative prediction), overall: {100*df_pred_neg_true / df_pred_neg_total:0.1f}% recidivism" )

Chance of recidivating for negative prediction (ie, meaning of negative prediction), overall: 31.8% recidivism


### Chance of being wrongly predicted high-risk

### White

In [101]:
dfp = df[ (df["race"]=="white") ]
dfp

,race,outcome,prediction,count
4,white,0,0,1139
5,white,0,1,349
6,white,1,0,461
7,white,1,1,505


In [102]:
TN = dfp[(dfp["outcome"]==0) & (dfp["prediction"]==0)].iloc[0]["count"]
FN = dfp[(dfp["outcome"]==1) & (dfp["prediction"]==0)].iloc[0]["count"]
TP = dfp[(dfp["outcome"]==1) & (dfp["prediction"]==1)].iloc[0]["count"]
FP = dfp[(dfp["outcome"]==0) & (dfp["prediction"]==1)].iloc[0]["count"]

In [103]:
all_neg = FP+TN

In [105]:
print( f"Chance of being wrongly predicted high-risk when not recidivating, white: {100*FP/all_neg:0.1f}%" )

Chance of being wrongly predicted high-risk when not recidivating, white: 23.5%


### Black

In [106]:
dfp = df[ (df["race"]=="black") ]
dfp

,race,outcome,prediction,count
0,black,0,0,990
1,black,0,1,805
2,black,1,0,532
3,black,1,1,1369


In [107]:
TN = dfp[(dfp["outcome"]==0) & (dfp["prediction"]==0)].iloc[0]["count"]
FN = dfp[(dfp["outcome"]==1) & (dfp["prediction"]==0)].iloc[0]["count"]
TP = dfp[(dfp["outcome"]==1) & (dfp["prediction"]==1)].iloc[0]["count"]
FP = dfp[(dfp["outcome"]==0) & (dfp["prediction"]==1)].iloc[0]["count"]

In [109]:
print( f"Chance of being wrongly predicted high-risk when not recidivating, black: {100*FP/all_neg:0.1f}%" )

Chance of being wrongly predicted high-risk when not recidivating, black: 54.1%


## Problem: unequal base rates

Recidivism is higher in the black subgroup.

In [126]:
df[(df["race"] == "black") & (df["outcome"] == 1)]["count"].sum() / df[(df["race"] == "black")]["count"].sum()

0.5143398268398268

In [127]:
df[(df["race"] == "white") & (df["outcome"] == 1)]["count"].sum() / df[(df["race"] == "white")]["count"].sum()

0.39364303178484106

It is impossible to equalize accuracy, calibration, and false positive rate at the same time if base rates are not equal.

In face, base rates are unequal because recidivism is a consequence of poverty, which is disproportionately effects Black people in the United States as an ongoing legacy of slavery and racialized classism.

There is no magical mathematical wand to undo a crime; algorithms will always be unfair as long as society is unfair.